## Naive Bayesian Classifier

In [1]:
from tqdm import tqdm
from konlpy import tag # 다양한 한국어 형태소 분석기 클래스가 담겨있다
from collections import defaultdict

import math

In [2]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."
]
train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

test_data = [
  "정말 좋았습니다. 또 가고 싶네요.",
  "별로였습니다. 되도록 가지 마세요.",
  "다른 분들께도 추천드릴 수 있을 만큼 만족했습니다.",
  "서비스가 좀 더 개선되었으면 좋겠습니다. 기분이 좀 나빴습니다."
]

In [3]:
tokenizer = tag.Okt()

def make_tokenized(data):
    tokenized = []
    for sent in tqdm(data):
        tokens = tokenizer.morphs(sent)
        tokenized.append(tokens)
    return tokenized


In [4]:
train_tokenized = make_tokenized( train_data )
test_tokenized = make_tokenized( test_data )
print(train_tokenized)

100%|██████████| 4/4 [00:00<00:00, 151.78it/s]

[['정말', '맛있습니다', '.', '추천', '합니다', '.'], ['기대했던', '것', '보단', '별로', '였네요', '.'], ['다', '좋은데', '가격', '이', '너무', '비싸서', '다시', '가고', '싶다는', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '입니다', '!', '재', '방문', '의사', '있습니다', '.'], ['음식', '도', '서비스', '도', '다', '만족스러웠습니다', '.'], ['위생', '상태', '가', '좀', '별로', '였습니다', '.', '좀', '더', '개선', '되', '기를', '바랍니다', '.'], ['맛', '도', '좋았고', '직원', '분들', '서비스', '도', '너무', '친절했습니다', '.'], ['기념일', '에', '방문', '했는데', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋았습니다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짰습니다', '.', '저', '는', '별로', '였네요', '.'], ['위생', '에', '조금', '더', '신경', '썼으면', '좋겠습니다', '.', '조금', '불쾌했습니다', '.']]


#### 단어 counts

In [5]:
word_count = defaultdict(int) # collections 활용

for tokens in tqdm(train_tokenized):
    for token in tokens:
        word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 66260.73it/s]


In [6]:
word_count = sorted( word_count.items(), key = lambda x : x[1], reverse = True)
print(len(word_count))

66


#### 단어, token별 idx부여

In [7]:
w2i = {}
for pair in tqdm(word_count):
    if pair[0] not in w2i:
        w2i[pair[0]] = len(w2i)
print(w2i) # 단어마다 개별 index를 부어

100%|██████████| 66/66 [00:00<00:00, 133602.35it/s]

{'.': 0, '도': 1, '별로': 2, '다': 3, '이': 4, '너무': 5, '음식': 6, '서비스': 7, '였네요': 8, '방문': 9, '위생': 10, '좀': 11, '더': 12, '에': 13, '조금': 14, '정말': 15, '맛있습니다': 16, '추천': 17, '합니다': 18, '기대했던': 19, '것': 20, '보단': 21, '좋은데': 22, '가격': 23, '비싸서': 24, '다시': 25, '가고': 26, '싶다는': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '입니다': 34, '!': 35, '재': 36, '의사': 37, '있습니다': 38, '만족스러웠습니다': 39, '상태': 40, '가': 41, '였습니다': 42, '개선': 43, '되': 44, '기를': 45, '바랍니다': 46, '맛': 47, '좋았고': 48, '직원': 49, '분들': 50, '친절했습니다': 51, '기념일': 52, '했는데': 53, '분위기': 54, '좋았습니다': 55, '전반': 56, '적': 57, '으로': 58, '짰습니다': 59, '저': 60, '는': 61, '신경': 62, '썼으면': 63, '좋겠습니다': 64, '불쾌했습니다': 65}


<br></br><br></br>

## Naive Bayes 구현하기

- self.k : Smoothing을 위한 상수.
- self.w2i : 사전에 구한 vocab
- self.priors : 각 class 의 사전 확률
- self.likelihoods : 각 token의 특정 조건 내의 likelihood..?

- smoothing을 위해서 라플라스 스무딩 수행

In [14]:
class NaiveBayesClassifier():
    def __init__(self, w2i, k=0.1):
        self.k = k
        self.w2i = w2i
        self.priors = {}
        self.likelihoods = {}
        
    def train( self, train_tokenized, train_labels ):
        self.set_priors(train_labels) 
        self.set_likelihoods(train_tokenized, train_labels)
        
    def inference(self, tokens):
        log_prob0 = 0.0
        log_prob1 = 0.0
        
        for token in tokens:
            if token in self.likelihoods:
                log_prob0 += math.log(self.likelihoods[token][0])
                log_prob1 += math.log(self.likelihoods[token][1])
        
        log_prob0 += math.log(self.priors[0])
        log_prob1 += math.log(self.priors[1])
        
        
        if log_prob0 >= log_prob1:
            return 0
        else:
            return 1
        
    def set_priors(self, train_labels):
        class_counts = defaultdict(int)
        
        for label in tqdm(train_labels):
            class_counts[label] += 1
        for label, count in class_counts.items():
            self.priors[label] = class_counts[label] / len(train_labels)
            
            
    def set_likelihoods(self, train_tokenized, train_labels):
        token_dists = {}
        class_counts = defaultdict(int)
        
        for i, label in enumerate(tqdm(train_labels)):
            count = 0
            for token in train_tokenized[i]:
                if token in self.w2i:
                    if token not in token_dists:
                        token_dists[token] = {0:0,1:0}
                    token_dists[token][label] += 1
                    count += 1
            class_counts[label] += count
        
        for token,dist in tqdm(token_dists.items()):
            if token not in self.likelihoods:
                self.likelihoods[token] = {
                    0:(token_dists[token][0] + self.k) / (class_counts[0] + len(self.w2i)*self.k),
                    1:(token_dists[token][1] + self.k) / (class_counts[1] + len(self.w2i)*self.k),
                }
    

In [15]:
classifier = NaiveBayesClassifier(w2i)
classifier.train(train_tokenized, train_labels)

100%|██████████| 66/66 [00:00<00:00, 399457.52it/s]


In [16]:
preds = []
for test_tokens in tqdm(test_tokenized):
    pred = classifier.inference(test_tokens)
    preds.append(pred)

100%|██████████| 4/4 [00:00<00:00, 30727.50it/s]
